In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf           

In [2]:
data = pd.read_csv('d:\\project\\train.csv')

In [3]:
images = data.iloc[:,1:].values
images = images.astype(np.float)
images = np.multiply(images, 1.0 / 255.0)

image_size = images.shape[1]
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

In [4]:
labels_flat = data[[0]].values.ravel()
labels_count = np.unique(labels_flat).shape[0]


In [5]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)

In [6]:
train_images = images[:]
train_labels = labels[:]

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
x = tf.placeholder('float', shape=[None, image_size])
y_ = tf.placeholder('float', shape=[None, labels_count])

In [10]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

image = tf.reshape(x, [-1,image_width , image_height,1])

h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)

h_pool1 = max_pool_2x2(h_conv1)

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

h_pool2 = max_pool_2x2(h_conv2)

In [12]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [13]:
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [14]:
W_fc2 = weight_variable([1024, labels_count])
b_fc2 = bias_variable([labels_count])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [15]:

cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [16]:
predict = tf.argmax(y,1)

In [17]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        epochs_completed += 1
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [18]:
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [19]:
train_accuracies = []
x_range = []

display_step=1

for i in range(10000):

    batch_xs, batch_ys = next_batch(50)        

    if i%display_step == 0 or (i+1) == 10000:
        
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       

        print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        if i%(display_step*10) == 0 and i:
            display_step *= 10

    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})

training_accuracy => 0.1000 for step 0
training_accuracy => 0.1400 for step 1
training_accuracy => 0.1200 for step 2
training_accuracy => 0.2200 for step 3
training_accuracy => 0.1800 for step 4
training_accuracy => 0.3200 for step 5
training_accuracy => 0.2200 for step 6
training_accuracy => 0.2000 for step 7
training_accuracy => 0.2000 for step 8
training_accuracy => 0.1600 for step 9
training_accuracy => 0.2200 for step 10
training_accuracy => 0.4600 for step 20
training_accuracy => 0.5800 for step 30
training_accuracy => 0.7000 for step 40
training_accuracy => 0.7600 for step 50
training_accuracy => 0.6400 for step 60
training_accuracy => 0.8000 for step 70
training_accuracy => 0.8800 for step 80
training_accuracy => 0.8400 for step 90
training_accuracy => 0.8000 for step 100
training_accuracy => 0.9000 for step 200
training_accuracy => 0.9400 for step 300
training_accuracy => 0.9600 for step 400
training_accuracy => 0.9200 for step 500
training_accuracy => 0.9400 for step 600
trai

In [20]:
test_images = pd.read_csv('d:\\project\\test.csv').values
test_images = test_images.astype(np.float)

test_images = np.multiply(test_images, 1.0 / 255.0)

predicted_lables = np.zeros(test_images.shape[0])
for i in range(0,test_images.shape[0]//50):
    predicted_lables[i*50 : (i+1)*50] = predict.eval(feed_dict={x: test_images[i*50 : (i+1)*50], 
                                                                                keep_prob: 1.0})
    
np.savetxt('d:\\submission3.csv', 
           np.c_[range(1,len(test_images)+1),predicted_lables], 
           delimiter=',', 
           header = 'Imageid,Label', 
           comments = '', 
           fmt='%d')